In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm

In [2]:
# 데이터 로드
path = "../stat_proj/data/prepro_test_data_ver.2.3.csv"
df = pd.read_csv(path)

#df = pd.read_csv('preprocessed_data_ver.2.3._IRR+LOG+BINARY.csv')

In [3]:
# 모든 열 다 보기
pd.set_option('display.max_columns', None)

# 모든 행 다 보기
pd.set_option('display.max_rows', None)

# 열 너비 넉넉하게
pd.set_option('display.width', None)

# 열 안의 값 자르지 않게
pd.set_option('display.max_colwidth', None)

In [4]:
df.head()

,funded_amnt,term,installment,emp_length,annual_inc,issue_d,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,recoveries,collection_recovery_fee,last_pymnt_d,last_fico_range_high,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,default,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,addr_state_nan,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,purpose_nan,emp_length_missing,percent_bc_gt_75_missing,collection_recovery_fee_missing,revol_bal_missing,open_il_12m_missing,open_act_il_missing,fico_range_high_missing,all_util_missing,mths_since_last_delinq_missing,acc_now_delinq_missing,open_acc_missing,pub_rec_missing,fico_range_low_missing,mths_since_recent_revol_delinq_missing,delinq_amnt_missing,funded_amnt_missing,delinq_2yrs_missing,annual_inc_missing,recoveries_missing,open_il_24m_missing,total_acc_missing,inq_last_12m_missing,open_rv_12m_missing,last_fico_range_high_missing,total_bal_il_missing,open_acc_6m_missing,open_rv_24m_missing,max_bal_bc_missing,installment_missing,mths_since_rcnt_il_missing,il_util_missing,mths_since_recent_inq_missing
0,20000.0,36,608.72,6.0,11.736077,2013-03-01,2.604909,0,785.0,789.0,1.0,0.0,12.0,0,9.559094,26.2,42.0,0.00,0.0000,Mar-2016,724.0,0,0,0,291526.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.896758,0.0,5.0,10.185013,10.486290,26.9,0,0.0,188.0,165.0,14.0,5.0,7.0,14.0,6.0,0.0,0.0,6.0,6.0,20.0,9.0,8.0,26.0,6.0,12.0,0,0,0,1.0,100.0,0.0,0,0,12.780957,10.542865,10.799596,10.740410,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,1,1,1,1,0,1,1,0
1,12000.0,60,283.22,0.5,11.002117,2014-03-01,2.357073,0,660.0,664.0,0.0,37.0,15.0,1,9.370075,45.5,36.0,1753.61,315.6498,Jun-2015,624.0,0,0,1,14331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.158169,0.0,7.0,6.932448,8.028781,73.1,0,0.0,123.0,201.0,8.0,8.0,4.0,20.0,8.0,37.0,2.0,9.0,6.0,17.0,2.0,13.0,27.0,9.0,14.0,0,0,0,2.0,87.5,66.7,1,0,10.349455,9.570250,9.341456,8.601534,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [5]:
# 독립변수에서 제외할 변수 목록
drop_cols = [
    'term', 'last_pymnt_d', 'installment', 'funded_amnt',
    'recoveries', 'collection_recovery_fee', 'default', 'issue_d'
]

X = df.drop(columns=drop_cols)
y = df['default']

In [6]:
from fredapi import Fred

# FRED API 연결
fred = Fred(api_key="f9ef939a9a1eeff85d31ed2c984507d9")  # 🔑 실제 API 키로 대체해야 함

# 미국 3년 만기 국채 수익률(GS3) 시계열 불러오기
series = fred.get_series('GS3')  # pandas.Series 형식으로 반환됨


print(series.head())

1953-04-01    2.51
1953-05-01    2.72
1953-06-01    2.74
1953-07-01    2.62
1953-08-01    2.58
dtype: float64


In [7]:
series.shape

(868,)

In [8]:
def get_nearest_rate(issue_date, rate_series):
    if pd.isnull(issue_date):
        return np.nan
    try:
        idx = rate_series.index.get_indexer([issue_date], method='nearest')[0]
        return rate_series.iloc[idx] / 100  # % 단위 → 소수로 변환
    except Exception as e:
        print(f"Error: {issue_date} ▶ {e}")
        return np.nan


df['risk_free_rate'] = df['issue_d'].apply(lambda x: get_nearest_rate(x, series))

In [9]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])
df['issue_d'] = pd.to_datetime(df['issue_d'])  # 혹시 모르니 이것도

/var/folders/2q/_8m2fvxd4pv04q2b3gk8j0bm0000gn/T/ipykernel_41849/4242469769.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])


In [10]:
print(df[['issue_d', 'risk_free_rate', 'last_pymnt_d']].head())

     issue_d  risk_free_rate last_pymnt_d
0 2013-03-01          0.0039   2016-03-01
1 2014-03-01          0.0082   2015-06-01
2 2016-02-01          0.0090   2018-03-01
3 2013-08-01          0.0070   2014-06-01
4 2015-04-01          0.0087   2017-05-01


In [11]:
from dateutil.relativedelta import relativedelta

def create_cash_flow_from_dates(row):
    try:
        term = int(row['term'])
        default = int(row['default'])
        issue_d = row['issue_d']  # 이미 datetime 형식
        last_pymnt_d = pd.to_datetime(row['last_pymnt_d'])  # 이건 문자열일 수도 있으니 변환
        installment = float(row['installment'])
        funded_amnt = float(row['funded_amnt'])
        recoveries = float(row['recoveries'])
        collection_fee = float(row['collection_recovery_fee'])

        # 첫 현금흐름: 대출 실행
        cash_flow = [-funded_amnt]

        if pd.isnull(issue_d) or pd.isnull(last_pymnt_d):
            return np.nan

        # 몇 회차까지 납입했는지 계산
        delta = relativedelta(last_pymnt_d, issue_d)
        last_pymnt_num = delta.years * 12 + delta.months

        for month in range(1, term + 1):
            if default == 1:
                if month <= last_pymnt_num:
                    cash_flow.append(installment)
                elif month == last_pymnt_num + 1:
                    cash_flow.append(recoveries - collection_fee)
                else:
                    cash_flow.append(0)
            else:
                cash_flow.append(installment)

        return cash_flow

    except Exception as e:
        print(f"[오류] index={row.name}, error={e}")
        return np.nan


In [12]:
df['cash_flow'] = df.apply(create_cash_flow_from_dates, axis=1)

In [13]:
!pip install numpy-financial


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [14]:
import numpy_financial as npf

def get_irr(cash_flow):
    if not isinstance(cash_flow, list) or len(cash_flow) == 0:
        return np.nan
    irr_monthly = npf.irr(cash_flow)
    if irr_monthly is None or np.isnan(irr_monthly):
        return np.nan
    return (1 + irr_monthly) ** 12 - 1  # 연환산

In [15]:
df['irr'] = df['cash_flow'].apply(get_irr)

In [16]:
df['irr'] = df['irr'].fillna(df['risk_free_rate'])

df['irr'] = df['cash_flow'].apply(lambda x: np.irr(x) if isinstance(x, list) else np.nan)




---



In [19]:
# 결과 저장용 리스트
best_models = []
best_thresholds = []
validation_sharpes = []
test_sharpes = []
test_approval_rates = []
test_irr_means = []
test_irr_positive_rates = []

In [20]:
# Sharpe 계산 함수
def calculate_sharpe(returns, risk_free_rates):
    excess = returns - risk_free_rates
    if excess.std(ddof=1) == 0:
        return -np.inf
    return excess.mean() / excess.std(ddof=1)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import json
import os

# 저장 경로 설정
SAVE_DIR = "../stat_proj"
os.makedirs(SAVE_DIR, exist_ok=True)

In [34]:
# 랜덤포레스트 하이퍼파라미터 범위 지정
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# 전체 데이터의 일부만 샘플링하여 튜닝 데이터 구성
X_tune_full, _, y_tune_full, _ = train_test_split(X, y, test_size=0.2, stratify=y)
X_tune, _, y_tune, _ = train_test_split(
    X_tune_full, y_tune_full,
    train_size=50000,  # 샘플 수 조절 가능
    stratify=y_tune_full,
    random_state=42
)

# RandomizedSearchCV 수행
model_base = RandomForestClassifier(random_state=42)
search = RandomizedSearchCV(
    model_base,
    param_distributions=param_dist,
    n_iter=5,
    scoring='roc_auc',
    cv=2,
    n_jobs=-1,
    verbose=2
)
search.fit(X_tune, y_tune)

# 최적 하이퍼파라미터 출력
best_params = search.best_params_
print("✅ Best hyperparameters (from tuning):", best_params)

Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV] END max_depth=5, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   1.5s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   1.6s
[CV] END max_depth=7, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   1.8s
[CV] END max_depth=7, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50; total time=   1.8s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   4.4s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total

In [39]:
best_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 10,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'n_jobs': -1
}

# 최적 하리퍼파라미터 결과
#best_params = {
#    'n_estimators': 300, 
#    'min_samples_split': 10, 
#    'min_samples_leaf': 4, 
#    'max_features': 'sqrt', 
#    'max_depth': 10
#}

In [40]:
# 데이터 전처리 - 무한대 값과 너무 큰 값 처리
print("=== 데이터 전처리 시작 ===")
print("원본 데이터 형태:", X.shape)

# 무한대 값 처리
X = X.replace([np.inf, -np.inf], np.nan)
print("무한대 값 처리 완료")

# NaN 값 처리
X = X.fillna(X.median())
print("NaN 값 처리 완료")

# 너무 큰 값 제한
X = X.clip(-1e6, 1e6)
print("큰 값 제한 완료")

print("=== 전처리 완료 ===")
print("최종 데이터 형태:", X.shape)
print("무한대 값:", np.isinf(X.select_dtypes(include=[np.number])).sum().sum())
print("NaN 값:", X.isnull().sum().sum())

=== 데이터 전처리 시작 ===
원본 데이터 형태: (744608, 172)
무한대 값 처리 완료
NaN 값 처리 완료
큰 값 제한 완료
=== 전처리 완료 ===
최종 데이터 형태: (744608, 172)
무한대 값: 0
NaN 값: 0


In [41]:
# 결과 저장용 리스트
best_models = []
best_thresholds = []
validation_sharpes = []
test_sharpes = []
test_approval_rates = []
test_irr_means = []
test_irr_positive_rates = []

# 베스트 모델 저장 변수
best_model = None
best_sharpe_overall = -np.inf
best_model_index = None
best_threshold_overall = None
best_params_overall = None

In [42]:
for i in tqdm(range(100)):
    # 매 반복마다 다른 seed 사용
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=i, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=i, stratify=y_temp
    )

    # 모델 학습
    model = RandomForestClassifier(**best_params, random_state=i)
    model.fit(X_train, y_train)

    # 검증셋 예측 확률
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    thresholds = np.arange(0.0, 1.0, 0.05)

    best_sharpe = -np.inf
    best_threshold = None
    df_val = df.loc[X_val.index]

    for threshold in thresholds:
        approved_mask = y_pred_proba <= threshold
        denied_mask = ~approved_mask

        selected = df_val.copy()
        selected.loc[approved_mask, 'irr_adj'] = selected.loc[approved_mask, 'irr']
        selected.loc[denied_mask, 'irr_adj'] = selected.loc[denied_mask, 'risk_free_rate']

        returns = selected['irr_adj']
        risk_free = selected['risk_free_rate']
        valid = returns.notnull() & risk_free.notnull()

        if valid.sum() < 2:
            continue

        sharpe = calculate_sharpe(returns[valid], risk_free[valid])

        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_threshold = threshold

    best_models.append(model)
    best_thresholds.append(best_threshold)
    validation_sharpes.append(best_sharpe)

    # 테스트셋 평가
    y_test_proba = model.predict_proba(X_test)[:, 1]
    test_approved_mask = y_test_proba <= best_threshold
    df_test = df.loc[X_test.index]
    test_selected = df_test[test_approved_mask]

    returns_test = test_selected['irr']
    risk_free_test = test_selected['risk_free_rate']
    valid = returns_test.notnull() & risk_free_test.notnull()
    returns_test = returns_test[valid]
    risk_free_test = risk_free_test[valid]

    sharpe_test = calculate_sharpe(returns_test, risk_free_test)
    test_sharpes.append(sharpe_test)
    test_approval_rates.append(len(returns_test) / len(df_test))
    test_irr_means.append(returns_test.mean())
    test_irr_positive_rates.append((returns_test > 0).mean())

    # Best 모델 업데이트
    if sharpe_test > best_sharpe_overall:
        best_sharpe_overall = sharpe_test
        best_model = model
        best_model_index = i
        best_threshold_overall = best_threshold
        best_params_overall = model.get_params()

# Best 모델 저장
joblib.dump(best_model, f"{SAVE_DIR}/best_mode2l.pkl")
with open(f"{SAVE_DIR}/best_model_threshold2.json", "w") as f:
    json.dump({"threshold": best_threshold_overall}, f)
with open(f"{SAVE_DIR}/best_model_params2.json", "w") as f:
    json.dump(best_params_overall, f)

# 최종 결과 출력
print("\n최종 결과 요약")
print("✅ Best model index:", best_model_index)
print("✅ Best validation Sharpe ratio:", validation_sharpes[best_model_index])
print("✅ Best test Sharpe ratio:", test_sharpes[best_model_index])
print("✅ Best approval rate:", test_approval_rates[best_model_index])
print("✅ Mean IRR:", test_irr_means[best_model_index])
print("✅ Positive IRR ratio:", test_irr_positive_rates[best_model_index])
print("✅ Best threshold:", best_thresholds[best_model_index])
print("✅ Best model params:", best_model.get_params())

  8%|▊         | 8/100 [03:13<44:13, 28.84s/it]Exception ignored in: <function ResourceTracker.__del__ at 0x104365bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x102d55bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ a


최종 결과 요약
✅ Best model index: 68
✅ Best validation Sharpe ratio: 0.5229762266856239
✅ Best test Sharpe ratio: 0.7706817651205201
✅ Best approval rate: 0.6139724150897785
✅ Mean IRR: 0.10998345044558736
✅ Positive IRR ratio: 0.9792199838134611
✅ Best threshold: 0.15000000000000002
✅ Best model params: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 68, 'verbose': 0, 'warm_start': False}


In [43]:
print("Best thresholds from 10 runs:")
print(best_thresholds)

print("\nValidation Sharpe Ratios from 10 runs:")
print(validation_sharpes)

print("\nTest Sharpe Ratios from 10 runs:")
print(test_sharpes)

print("\nTest Approval Rates:")
print(test_approval_rates)

print("\nTest IRR Means:")
print(test_irr_means)

print("\nTest IRR Positive Rates:")
print(test_irr_positive_rates)

Best thresholds from 10 runs:
[np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.2), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.150000000000000